In [14]:
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.utilities import WikipediaAPIWrapper

import json


def DuckDuckGoSearch(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    query = inputs["query"]
    return ddg.run(query)


def WikipediaSearch(inputs):
    wikipedia = WikipediaAPIWrapper()
    query = inputs["query"]
    return wikipedia.run(query)

functions_map = {
    "DuckDuckGoSearch": DuckDuckGoSearch,
    "WikipediaSearch": WikipediaSearch,
}

functions = [
    {
        "type": "function",
        "function": {
            "name": "DuckDuckGoSearch",
            "description": "Use this tool to search on DuckDuckGo.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The query you will search for",
                    }
                },
                "required": ["query"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "WikipediaSearch",
            "description": "Use this tool to search on Wikipedia.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The query you will search for",
                    }
                },
                "required": ["query"],
            },
        },
    },
]

In [15]:
import openai as client

# assistant = client.beta.assistants.create(
#     name="Research Assistant",
#     instructions="The agent attempts to search DuckDuckGo and Wikipedia, extracts the content, and then provides the research to you.",
#     model="gpt-3.5-turbo",
#     tools=functions,
# )
assistant_id = "asst_TMRasTUZsY9P46mtYen2q4Nv"

In [16]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "Research about the XZ backdoor",
        }
    ]
)
#thread

In [17]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
#run

In [18]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")


def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread_id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id,
            }
        )
    return outputs


def submit_tool_outputs(run_id, thread_id):
    outpus = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id,
        thread_id=thread_id,
        tool_outputs=outpus,
    )

In [22]:
get_run(run.id, thread.id).status

'completed'

In [21]:
submit_tool_outputs(run.id, thread.id)

Calling function: DuckDuckGoSearch with arg {"query": "XZ backdoor"}
Calling function: WikipediaSearch with arg {"query": "XZ backdoor"}


Run(id='run_sYNXYEgkiMC98XbjlVPKeCDD', assistant_id='asst_TMRasTUZsY9P46mtYen2q4Nv', cancelled_at=None, completed_at=None, created_at=1713112583, expires_at=1713113183, failed_at=None, file_ids=[], instructions='The agent attempts to search DuckDuckGo and Wikipedia, extracts the content, and then provides the research to you.', last_error=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=None, started_at=1713112583, status='queued', thread_id='thread_NIHGXtYnB4aHsuVh8WyrErvw', tools=[FunctionTool(function=FunctionDefinition(name='DuckDuckGoSearch', description='Use this tool to search on DuckDuckGo.', parameters={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'The query you will search for'}}, 'required': ['query']}), type='function'), FunctionTool(function=FunctionDefinition(name='WikipediaSearch', description='Use this tool to search on Wikipedia.', parameters={'type': 'object', 'properties': {'query': {'type': 'string', 'desc

In [23]:
get_messages(thread.id)

user: Research about the XZ backdoor
assistant: ### DuckDuckGo Search Results:
- Malicious code added to xz Utils versions 5.6.0 and 5.6.1 modified the software's functionality, manipulating sshd (executable file for remote connections).
- Backdoor vulnerability (CVE-2024-3094) affects versions 5.6.0 and 5.6.1; organizations advised to revert to a non-compromised version like 5.4.
- Discovered by Microsoft developer Andres Freund, impacting widely used Linux distributions (Red Hat, Debian).

### Wikipedia Search Results:
- **XZ Utils Backdoor**: Discovered on March 29, 2024, by Andres Freund in xz utility, liblzma library versions 5.6.0 and 5.6.1.
- **Backdoor Description**: Covert method to bypass authentication/encryption, allowing remote code execution; Assigned CVE-2024-3094 with a CVSS score of 10.0.
- **Backdoor Definition**: A hidden method of bypassing security measures, covertly accessing systems or data for malicious purposes.

Would you like more information on any specific 